In [ ]:
!pip install pytube torch torchvision torchaudio openai-whisper transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 954.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Usin

In [ ]:
import os
from pytube import YouTube
import whisper
import torch

# Download YouTube Video and Extract Audio
def download_youtube_audio(url, output_path='audio.mp4'):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    audio_file = video.download(filename=output_path)
    return audio_file

# Convert Speech to Text using Whisper
def speech_to_text(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result['text']

# Example usage
if __name__ == "__main__":
    youtube_url = 'https://youtu.be/pMX2cQdPubk?si=hpEW6TGAacyPkaVT'
    audio_path = download_youtube_audio(youtube_url)
    transcript = speech_to_text(audio_path)
    print("Transcript:", transcript)


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 104MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcript:  Alright, Tim. Great to see you. Thanks for spending the time. Thank you for coming. So you have the WWDC keynote today. Yeah. Which was really fun to watch as a whole two hours. I want to zoom all the way out, because obviously a lot of talk about AI in general. Yeah. And I'm kind of just left wondering how Apple defines AI in general. Because I know if you ask a regular person, you might hear about generative AI, chat bots, things like that. And these are relatively new additions to Apple's AI portfolio. How do you look at AI as Apple? Well, we've been executing with AI for a long time. Right. It's that you're wearing a watch. It's at the root of the watch. I mean, you think about things like crash detection, fall detection, things like a fib, and all of this kind of stuff is machine learning at the end of the day. And so, but what has captured people's imagination is generative AI. And we see it as the opportunity for a whole new curve of technology and providing, and do

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, EncoderDecoderModel, BertTokenizer

def summarize_text(text, model_name="facebook/bart-large-cnn"):
    if model_name.startswith("facebook/bart"):
        tokenizer = BartTokenizer.from_pretrained(model_name)
        model = BartForConditionalGeneration.from_pretrained(model_name)
        inputs = tokenizer(text, max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = model.generate(
            inputs['input_ids'],
            max_length=400,
            min_length=200,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    elif model_name.startswith("bert"):
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
        inputs = tokenizer(text, max_length=512, return_tensors='pt', truncation=True)
        summary_ids = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=400,
            min_length=200,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    else:
        raise ValueError("Unsupported model_name. Choose 'facebook/bart-' or 'bert-'")

    return summary

if __name__ == "__main__":

    summary_bart = summarize_text(transcript, model_name="facebook/bart-large-cnn")
    print("Summary using BART:")
    points = summary_bart.split(". ")
    for point in points:
        print(point)

    summary_bertsum = summarize_text(transcript, model_name="bert-base-uncased")
    print("\nSummary using BERTSum:")
    points = summary_bertsum.split(". ")
    for point in points:
        print(point)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]


Summary using BERTSum:
the wwdc keynote was held at the root of the watch
it was the first time that apple has been able to define ai in general
it's the opportunity for a whole new curve of technology and providing an assistant for people to provide an assistant to people
the benefit to the user is crash detection and fall detection, not the technology behind the feature, says tim cook
we're actually branding it apple intelligence now, and we've been working on it for a long time
we have always been focused on privacy and privacy, he says
we are branding it as apple intelligence, but we are not do not want it to be a tool for the user, he adds
we should not be talking about it as a tool that lets it works very carefully, we'll be able to be available for everyone's benefit, he tells mailonline
if we's not a tool, we should have to follow it up
we need to make it easier to use it.
